In [ ]:
import numpy as np
import torch
import torch.nn as nn
from torch.utils.data import DataLoader, WeightedRandomSampler
from sklearn.model_selection import train_test_split
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import f1_score, precision_score, recall_score
import torch.optim as optim
from torch.utils.data import Dataset
from torch.amp import autocast, GradScaler
from torchvision import transforms, datasets
from PIL import Image
from tqdm import tqdm

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!ls /content/drive/MyDrive/Colab_Notebooks/GAIDI/Deepfake/


data				  manipulated
downloaded_files_log.txt	  originals
faceforensic_downloader_colab.py  Pytorch_Training.ipynb
Frame_Extraction.ipynb


In [ ]:
# zip the files in colab on CPU
# !cd /content/drive/MyDrive/Colab_Notebooks/GAIDI/Deepfake/data && zip -r /content/drive/MyDrive/zipped_frames.zip ./*

Streaming output truncated to the last 5000 lines.
  adding: real/761.mp4_0003.jpg (deflated 1%)
  adding: real/761.mp4_0004.jpg (deflated 1%)
  adding: real/761.mp4_0005.jpg (deflated 1%)
  adding: real/761.mp4_0006.jpg (deflated 1%)
  adding: real/761.mp4_0007.jpg (deflated 1%)
  adding: real/761.mp4_0008.jpg (deflated 1%)
  adding: real/761.mp4_0009.jpg (deflated 1%)
  adding: real/761.mp4_0010.jpg (deflated 1%)
  adding: real/761.mp4_0011.jpg (deflated 1%)
  adding: real/761.mp4_0012.jpg (deflated 1%)
  adding: real/761.mp4_0013.jpg (deflated 1%)
  adding: real/761.mp4_0014.jpg (deflated 1%)
  adding: real/761.mp4_0015.jpg (deflated 1%)
  adding: real/761.mp4_0016.jpg (deflated 1%)
  adding: real/761.mp4_0017.jpg (deflated 1%)
  adding: real/761.mp4_0018.jpg (deflated 1%)
  adding: real/761.mp4_0019.jpg (deflated 1%)
  adding: real/761.mp4_0020.jpg (deflated 1%)
  adding: real/761.mp4_0021.jpg (deflated 1%)
  adding: real/761.mp4_0022.jpg (deflated 1%)
  adding: real/761.mp4_0023.j

In [ ]:
#  unzip directly into local directory (on A100)
!unzip /content/drive/MyDrive/zipped_frames_clean.zip -d /content/data

Streaming output truncated to the last 5000 lines.
  inflating: /content/data/real/761.mp4_0002.jpg  
  inflating: /content/data/real/761.mp4_0003.jpg  
  inflating: /content/data/real/761.mp4_0004.jpg  
  inflating: /content/data/real/761.mp4_0005.jpg  
  inflating: /content/data/real/761.mp4_0006.jpg  
  inflating: /content/data/real/761.mp4_0007.jpg  
  inflating: /content/data/real/761.mp4_0008.jpg  
  inflating: /content/data/real/761.mp4_0009.jpg  
  inflating: /content/data/real/761.mp4_0010.jpg  
  inflating: /content/data/real/761.mp4_0011.jpg  
  inflating: /content/data/real/761.mp4_0012.jpg  
  inflating: /content/data/real/761.mp4_0013.jpg  
  inflating: /content/data/real/761.mp4_0014.jpg  
  inflating: /content/data/real/761.mp4_0015.jpg  
  inflating: /content/data/real/761.mp4_0016.jpg  
  inflating: /content/data/real/761.mp4_0017.jpg  
  inflating: /content/data/real/761.mp4_0018.jpg  
  inflating: /content/data/real/761.mp4_0019.jpg  
  inflating: /content/data/real

# Torch Model

## Training modules

In [ ]:
## After splitting, we need a way to load images + labels from our lists of X and y — that’s where FrameDataset comes in.

class FrameDataset(Dataset):
    def __init__(self, image_paths, labels, transform = None):
        self.image_paths = image_paths
        self.labels = labels
        self.transform = transform

    def __len__(self):
        return len(self.image_paths)

    def __getitem__(self,idx):
        image = Image.open(self.image_paths[idx]).convert('RGB')
        label = self.labels[idx]

        if self.transform:
            image = self.transform(image)
        return image, label

In [ ]:
# Model
class DeepF_CNN(nn.Module):
    def __init__(self):
        super().__init__()
        self.net = nn.Sequential(
            # Block 1
            nn.Conv2d(in_channels=3, out_channels=32, kernel_size=3, padding=1),  # padding='same' → padding=1 when kernel=3
            nn.ReLU(),
            nn.BatchNorm2d(num_features=32),  # because output channels = 32
            nn.MaxPool2d(kernel_size=2),
            nn.Dropout(p=0.2),

            # Block 2
            nn.Conv2d(in_channels=32, out_channels=64, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.BatchNorm2d(num_features=64),
            nn.MaxPool2d(kernel_size=2),
            nn.Dropout(p=0.3),

            # Block 3
            nn.Conv2d(64, 128, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.BatchNorm2d(128),
            nn.AdaptiveAvgPool2d((1, 1)),  # → output shape [batch, 128, 1, 1]
            nn.Flatten(),                  # → [batch, 128]

            nn.Linear(128, 1)
            )
    def forward(self, x):
        return self.net(x)

In [ ]:
#Create ES function
class EarlyStopping:
    def __init__(self, patience=4):
        self.patience = patience
        self.counter = 0
        self.best_loss = float('inf')
        self.best_model = None
        self.early_stop = False

    def __call__(self, val_loss, model):
        if val_loss < self.best_loss:
            self.best_loss = val_loss
            self.best_model = model.state_dict() # saves best weight
            self.counter = 0
        else:
            self.counter += 1
            if self.counter >= self.patience:
                self.early_stop = True

# Prepare data

In [ ]:
from collections import Counter

transform = transforms.Compose([
    transforms.Resize((224,224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.5]*3, std=[0.5]*3)
])

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Using device:", device)
# skf = StratifiedKFold(n_splits=3, shuffle=True, random_state=42) # 3 fold TOO SLOW ON MY LAPTOP

# data = datasets.ImageFolder(root='/content/drive/MyDrive/Colab_Notebooks/GAIDI/Deepfake/data', transform=transform)
data = datasets.ImageFolder(root='/content/data', transform=transform) #  <== after copying the dataset to colab local disk with unzip
class_counts = Counter(data.targets)

print(f'data has two classes: {data.classes}, there are {len(data)} images(frames) in data, {class_counts[1]} real video frames, {class_counts[0]} fake video frames')

if ((class_counts[0] * 100) / class_counts[1]) < 45 or ((class_counts[0] * 100) / class_counts[1]) > 55:
    print('classes weights are imbalanced, WeightedRandomSampler is required')
else:
    print('classes weights are balanced, no WeightedRandomSampler required.')

Using device: cpu
data has two classes: ['fake', 'real'], there are 19061 images(frames) in data, 8332 real video frames, 10729 fake video frames
classes weights are imbalanced, WeightedRandomSampler is required


In [ ]:
X = np.array([s[0] for s in data.samples])
y = np.array([s[1] for s in data.samples])

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, stratify=y)

In [ ]:
print(f'{X_train.shape}\n{ X_val.shape}\n{y_train.shape}\n{y_val.shape}')

(15248,)
(3813,)
(15248,)
(3813,)


## 1st run

In [ ]:
def train_one_fold(X_train, y_train, X_val, y_val, transform, device, run):
    scaler = GradScaler()

    # Datasets
    train_dataset = FrameDataset(X_train, y_train, transform)
    val_dataset   = FrameDataset(X_val, y_val, transform)


    # Weighted sampler
    class_counts = np.bincount(y_train)
    weights = 1. / class_counts
    sample_weights = weights[y_train]
    sampler = WeightedRandomSampler(sample_weights, len(sample_weights), replacement=True)

    # DataLoaders
    train_loader = DataLoader(train_dataset, batch_size=128, sampler=sampler,
                              num_workers=2, pin_memory=True)
    val_loader = DataLoader(val_dataset, batch_size=128, shuffle=False,
                            num_workers=2, pin_memory=True)

    # Model, loss, optimizer
    model = DeepF_CNN().to(device)
    criterion = nn.BCEWithLogitsLoss()
    optimizer = optim.Adam(model.parameters(), lr=1e-3)
    scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', factor=0.5, patience=3)
    early_stopper = EarlyStopping(patience=5)

    for epoch in range(20):
        model.train()
        running_loss, correct, total = 0.0, 0, 0

        for images, labels in tqdm(train_loader, desc=f"Epoch {epoch+1}"):
            images = images.to(device, non_blocking=True)
            labels = labels.float().unsqueeze(1).to(device, non_blocking=True)

            optimizer.zero_grad()
            with autocast(device_type='cuda'):
                outputs = model(images)
                loss = criterion(outputs, labels)

            scaler.scale(loss).backward()
            scaler.step(optimizer)
            scaler.update()

            running_loss += loss.item() * labels.size(0)
            preds = torch.sigmoid(outputs) >= 0.5
            correct += (preds == labels).sum().item()
            total += labels.size(0)

        train_loss = running_loss / total
        train_acc = correct / total

        # Validation
        model.eval()
        val_loss, val_correct, val_total = 0.0, 0, 0
        all_preds, all_labels = [], []

        with torch.no_grad():
            for images, labels in val_loader:
                images = images.to(device, non_blocking=True)
                labels = labels.float().unsqueeze(1).to(device, non_blocking=True)

                with autocast(device_type='cuda'):
                    outputs = model(images)
                    loss = criterion(outputs, labels)

                val_loss += loss.item() * labels.size(0)
                preds = torch.sigmoid(outputs) >= 0.5
                val_correct += (preds == labels).sum().item()
                val_total += labels.size(0)

                all_preds.extend(preds.cpu().numpy())
                all_labels.extend(labels.cpu().numpy())

        val_loss /= val_total
        val_acc = val_correct / val_total
        f1 = f1_score(all_labels, all_preds)
        precision = precision_score(all_labels, all_preds)
        recall = recall_score(all_labels, all_preds)

        print(f"Epoch {epoch+1} | Train Loss: {train_loss:.4f} | Val Loss: {val_loss:.4f} | "
              f"Val Acc: {val_acc:.4f} | F1: {f1:.4f} | Precision: {precision:.4f} | Recall: {recall:.4f}")

        scheduler.step(val_loss)
        early_stopper(val_loss, model)

        if early_stopper.early_stop:
            print("Early stopping triggered.")
            break

    model.load_state_dict(early_stopper.best_model)
    torch.save({
    'epoch': epoch,
    'model_state_dict': model.state_dict(),
    'optimizer_state_dict': optimizer.state_dict(),
    'scheduler_state_dict': scheduler.state_dict(),
    'scaler_state_dict': scaler.state_dict(),  # AMP scaler
}, f'model_fold{run}.pth')

    return model, all_preds, all_labels


## Load model

In [ ]:
!ls -lh /content/model_fold1.pth

-rw-r--r-- 1 root root 1.1M Jul 16 07:49 /content/model_fold1.pth


In [ ]:
# Load saved checkpoint
checkpoint = torch.load('/content/model_fold1.pth', map_location=torch.device('cuda' if torch.cuda.is_available() else 'cpu'))


# Rebuild model, optimizer, scheduler, scaler (must match original setup)
model = DeepF_CNN().to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)
scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', factor=0.5, patience=3)
scaler = GradScaler()

# Load states
model.load_state_dict(checkpoint['model_state_dict'])
optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
scheduler.load_state_dict(checkpoint['scheduler_state_dict'])
scaler.load_state_dict(checkpoint['scaler_state_dict'])

# Determine where to continue
start_epoch = checkpoint['epoch'] + 1
n_more_epochs = 15

/usr/local/lib/python3.11/dist-packages/torch/amp/grad_scaler.py:132: UserWarning: torch.cuda.amp.GradScaler is enabled, but CUDA is not available.  Disabling.
  warnings.warn(


## Next runs

In [ ]:
def train_one_fold(X_train, y_train, X_val, y_val, transform, device, run):
    scaler = GradScaler()

    # Datasets
    train_dataset = FrameDataset(X_train, y_train, transform)
    val_dataset   = FrameDataset(X_val, y_val, transform)


    # Weighted sampler
    class_counts = np.bincount(y_train)
    weights = 1. / class_counts
    sample_weights = weights[y_train]
    sampler = WeightedRandomSampler(sample_weights, len(sample_weights), replacement=True)

    # DataLoaders
    train_loader = DataLoader(train_dataset, batch_size=128, sampler=sampler,
                              num_workers=2, pin_memory=True)
    val_loader = DataLoader(val_dataset, batch_size=128, shuffle=False,
                            num_workers=2, pin_memory=True)

    # Model, loss, optimizer
    model = DeepF_CNN().to(device)
    criterion = nn.BCEWithLogitsLoss()
    optimizer = optim.Adam(model.parameters(), lr=1e-3)
    scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', factor=0.5, patience=3)
    early_stopper = EarlyStopping(patience=5)

    for epoch in range(start_epoch, start_epoch + n_more_epochs):
        model.train()
        running_loss, correct, total = 0.0, 0, 0

        for images, labels in tqdm(train_loader, desc=f"Epoch {epoch+1}"):
            images = images.to(device, non_blocking=True)
            labels = labels.float().unsqueeze(1).to(device, non_blocking=True)

            optimizer.zero_grad()
            with autocast(device_type='cuda'):
                outputs = model(images)
                loss = criterion(outputs, labels)

            scaler.scale(loss).backward()
            scaler.step(optimizer)
            scaler.update()

            running_loss += loss.item() * labels.size(0)
            preds = torch.sigmoid(outputs) >= 0.5
            correct += (preds == labels).sum().item()
            total += labels.size(0)

        train_loss = running_loss / total
        train_acc = correct / total

        # Validation
        model.eval()
        val_loss, val_correct, val_total = 0.0, 0, 0
        all_preds, all_labels = [], []

        with torch.no_grad():
            for images, labels in val_loader:
                images = images.to(device, non_blocking=True)
                labels = labels.float().unsqueeze(1).to(device, non_blocking=True)

                with autocast(device_type='cuda'):
                    outputs = model(images)
                    loss = criterion(outputs, labels)

                val_loss += loss.item() * labels.size(0)
                preds = torch.sigmoid(outputs) >= 0.5
                val_correct += (preds == labels).sum().item()
                val_total += labels.size(0)

                all_preds.extend(preds.cpu().numpy())
                all_labels.extend(labels.cpu().numpy())

        val_loss /= val_total
        val_acc = val_correct / val_total
        f1 = f1_score(all_labels, all_preds)
        precision = precision_score(all_labels, all_preds)
        recall = recall_score(all_labels, all_preds)

        print(f"Epoch {epoch+1} | Train Loss: {train_loss:.4f} | Val Loss: {val_loss:.4f} | "
              f"Val Acc: {val_acc:.4f} | F1: {f1:.4f} | Precision: {precision:.4f} | Recall: {recall:.4f}")

        scheduler.step(val_loss)
        early_stopper(val_loss, model)

        if early_stopper.early_stop:
            print("Early stopping triggered.")
            break

    model.load_state_dict(early_stopper.best_model)
    torch.save({
    'epoch': epoch,
    'model_state_dict': model.state_dict(),
    'optimizer_state_dict': optimizer.state_dict(),
    'scheduler_state_dict': scheduler.state_dict(),
    'scaler_state_dict': scaler.state_dict(),  # AMP scaler
}, f'model_fold{run}.pth')

    return model, all_preds, all_labels


# Train

In [ ]:
model = train_one_fold(X_train, y_train, X_val, y_val, transform, device, run = 1)

Epoch 1: 100%|██████████| 120/120 [01:24<00:00,  1.42it/s]


Epoch 1 | Train Loss: 0.5857 | Val Loss: 0.5489 | Val Acc: 0.7097 | F1: 0.6345 | Precision: 0.7056 | Recall: 0.5765


Epoch 2: 100%|██████████| 120/120 [01:23<00:00,  1.44it/s]


Epoch 2 | Train Loss: 0.4426 | Val Loss: 0.3706 | Val Acc: 0.8434 | F1: 0.8116 | Precision: 0.8562 | Recall: 0.7714


Epoch 3: 100%|██████████| 120/120 [01:24<00:00,  1.42it/s]


...

Epoch 18 | Train Loss: 0.0596 | Val Loss: 0.1032 | Val Acc: 0.9588 | F1: 0.9536 | Precision: 0.9405 | Recall: 0.9670


Epoch 19: 100%|██████████| 120/120 [01:25<00:00,  1.41it/s]


Epoch 19 | Train Loss: 0.0711 | Val Loss: 0.0990 | Val Acc: 0.9565 | F1: 0.9511 | Precision: 0.9351 | Recall: 0.9676


Epoch 20: 100%|██████████| 120/120 [01:22<00:00,  1.45it/s]


Epoch 20 | Train Loss: 0.0548 | Val Loss: 0.0489 | Val Acc: 0.9738 | F1: 0.9698 | Precision: 0.9757 | Recall: 0.9640
